In [7]:
import numpy as np
import scipy.constants as cts
pcts = cts.physical_constants
from scipy.special import sph_harm

from scipy.integrate import quad as integral

In [9]:
def red_mass(m_1, m_2):
    return(m_1 * m_2 / (m_1 + m_2))

def energy_wl(x):
    return(cts.c * cts.h / x)

In [13]:
meV = cts.eV * 1e-3
eV = cts.eV
keV = cts.eV * 1e3
MeV = cts.eV * 1e6

### 1: Muonic atoms

Ba: $m = 137.7$ amu, $Z=56$, $m_\mu = 206 m_e$:

* Calculate the wavelength for the transition $n=2$ to $n=1$.
* Calculate the ionization energy
* Calculate the $n$ for which the radius would be the bohr radius
* What if instead of the muon we had a meson?

In [45]:
def bohr_energy(reduced_mass, Z, n):
    return(-1/2 * reduced_mass * (cts.c * cts.alpha * Z)**2 / (n**2))

muon_mass = pcts['muon mass'][0]
nucleus_mass = 137.3 * cts.value('atomic mass constant')
reduced_mass = red_mass(muon_mass, nucleus_mass)

Z = 56
energy_21 = bohr_energy(reduced_mass, Z, 2) - bohr_energy(reduced_mass, Z, 1)

mass_ratio = 1/pcts['electron-muon mass ratio'][0]
n = np.floor(np.sqrt(mass_ratio))

print(str(energy_21 / MeV) + "MeV")
print(str(-bohr_energy(reduced_mass, Z, 1) / MeV) + "MeV")
print(n)

6.611245219177223MeV
8.814993625569631MeV
14.0


Recall that:

$$
r_n = \left( \frac{4\pi \varepsilon_0}{e^2 m_e} \right) \frac{n^2}{Z}
$$

So the dependences on $n$ and $m_e$, which we are interested in, are to the powers 2 and -1 respectively.

Then, to account for the increase in $m_e$ we need to increase $n$.

### Infinite well perturbations

Take an infinite potential well. We know its eigenfunctions $\psi_n ^{(0)} (x) = \sqrt{2/a} \cos (\pi x / 2a)$. We want to perturb them with a constant potential addition to the Hamiltonian: $H_1 = V_1 [-a/2, a/2]$. (the brackets are [Iverson brackets](https://en.wikipedia.org/wiki/Iverson_bracket)).

* What will be the energy $E_1^{(1)}$?
* Could we also perturb if we had $V_1 > E^0_1$?

### Superfluid He

Our physical reference is a finite potential well, of depth 1eV, at temperature $T=2.17$K, therefore $kT=0.25$meV.

We insert an electron in the potential well, which polarizes it. The perturbation is modelled as

$$
V_p (x) = \begin{cases}
-\frac{2V_0}{R}x \qquad 0 \leq x \leq R/2 \\
\frac{2V_0}{R}(x-R) \qquad R/2 \leq x \leq R
\end{cases}
$$


Recall that the eigenfunctions for the infinite well can be written as:

$\psi_n^+ (x) = \cos ((n-1/2) \frac{\pi x}{a})$ and $\psi_n^- (x) = \sin (\frac{n\pi x}{a})$.

Their energies are $E^+_n = \frac{\hbar^2 \pi^2}{2ma^2} (n-1/2)^2$ and $E^-_n = \frac{\hbar^2 \pi^2}{2ma^2} (n)^2$.

* What will be the energy $E_1^1 = <1|H_1|1>$?
* What will be first nonzero $c_{k1}$ coefficient, $c_{13}^1 = <3|H_1|1> / (E_1^0 - E_3^0)$?

In [37]:
a = 1 # It is just a parameter, so we will not worry about it

def well_functions(x, a, n, parity):
    # a = half-length of well
    norm = np.sqrt(1/a)
    const = np.pi / a
    if(parity==1):
        return(norm * np.cos((n-1/2) *x*const))
    elif(parity==-1):
        return(norm * np.sin(n*x*const))

def well_energy(x, a, n, m, parity):
    const = (cts.hbar * np.pi / a)**2 / (2*m)
    if(parity==1):
        return(const * (n-1/2)**2)
    elif(parity==-1):
        return(const * n**2)

def perturbation(x, R, V_0):
    if(x<R/2):
        return(-2*V_0*x/R)
    else:
        return(2*V_0*(x-R)/R)

    
R = 1e-10
V_0 = 5/2 *meV
to_integrate = lambda x : well_functions(x, R/2, 1, -1)**2 * perturbation(x, R=R, V_0=V_0)
E11 = integral(to_integrate, 0, R)[0] / meV

In [36]:
E11

-1.2497849086818993